In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

挑战者 vs 受祝福方法在将模型部署到生产中的方法论

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/challenger_vs_blessed_deployment_method.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/challenger_vs_blessed_deployment_method.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/pipelines/official/challenger_vs_blessed_deployment_method.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                  
</table>

*注：此笔记本使用 KFP 1.x 和 GCPC 1.x。我们建议使用 2.x 版本*

请注意：此笔记本已在以下环境中测试过：

* Python 版本 = 3.9

## 概述

本教程展示了如何使用Vertex AI Pipeline通过挑战者vs受保护者方法将下一个版本的模型部署到生产环境中。

了解更多关于[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)和[Vertex AI中的模型评估](https://cloud.google.com/vertex-ai/docs/evaluation/introduction)。

### 目标

在本教程中，您将学习如何构建一个 Vertex AI 管道，该管道训练一个新的挑战者版本的模型，评估模型并将评估结果与生产中现有的祝福模型进行比较，以确定挑战者模型是否可以成为用于替换生产中的祝福模型。

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI 管道
- Vertex AI 模型评估
- Vertex AI 模型注册表
- Vertex AI 终端

执行的步骤包括：

- 将预训练（祝福的）模型导入到 `Vertex AI 模型注册表` 中。
- 将合成模型评估指标导入到相应（祝福的）模型中。
- 创建一个 `Vertex AI 终端` 资源。
- 将祝福模型部署到 `终端` 资源。
- 创建一个 Vertex AI 管道
    - 获取祝福的模型。
    - 导入另一个实例（挑战者）的预训练模型。
    - 将预训练（挑战者）模型注册为现有祝福模型的新版本。
    - 创建一个合成模型评估。
    - 将合成模型评估指标导入到相应挑战者模型中。
    - 比较评估结果并将祝福或挑战者设置为默认。
    - 部署新的祝福模型。

了解更多关于 [Vertex AI 管道](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)。

数据集

本教程使用的数据集是来自TensorFlow数据集的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已集成到TensorFlow中。训练模型预测图像属于十种类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船或卡车。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，
以及 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，
并使用 [价格计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

## 安装

安装以下必要的软件包以执行这个笔记本。

In [ ]:
# Install the packages
USER=''
! pip3 install {USER} --upgrade google-cloud-aiplatform \
                                'google-cloud-pipeline-components<2'
! pip3 install {USER}           tensorflow==2.5 \
                                tensorflow_hub
  
! pip3 install {USER} --upgrade 'kfp<2'

### 仅限Colab使用：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的 Google 云项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google 云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个账户时，您会得到 $300 的免费信用额度，可以用来支付计算/存储费用。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API]。

4. 如果您是在本地运行这个笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google云账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照下面的相关说明操作。

1. 顶点AI工作台
* 无需操作，因为您已经通过验证。

2. 本地的JupyterLab实例，请取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
IS_COLAB = False
# from google.colab import auth
# auth.authenticate_user()
# IS_COLAB = True

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务账户授予云存储权限。

创建一个云存储桶

创建一个存储桶，用于存储诸如数据集等中间产物。

- *{对笔记本作者的提示：对于任何需要是唯一的用户提供的字符串（比如存储桶名称或模型ID），请在末尾添加“-unique”，以便进行适当的测试}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 服务账号

您可以使用服务账号来创建Vertex AI Pipeline作业。

如果您不想使用项目的Compute Engine服务账号，请将`SERVICE_ACCOUNT`设置为另一个服务账号ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

设置Vertex AI管道的服务帐户访问权限

运行以下命令，授予您的服务帐户访问权限，以读取和写入在上一步中创建的存储桶中的管道工件 -- 您仅需要为每个服务帐户运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

导入库

In [ ]:
import kfp
import tensorflow as tf
import tensorflow_hub as hub
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic
from kfp.v2 import compiler
from kfp.v2.dsl import component

初始化用于Python的Vertex AI SDK，用于您的项目。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持 GPU 的容器映像，以及分配给虚拟机实例的 GPU 数量。例如，要使用包含 4 个 Nvidia Telsa K80 GPU 的 GPU 容器映像分配给每个 VM，您应该指定：

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，请指定`(None, None)`以使用在 CPU 上运行的容器映像。

了解更多关于[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

In [ ]:
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置预构建容器

为训练和预测设置预构建的Docker容器镜像。

有关最新列表，请参阅[预构建容器用于预测](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2.5".replace(".", "-")

if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 设置变量`DEPLOY_COMPUTE`来配置您将用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意: 以下内容不适用于训练:*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意: 您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

### 保存模型工件

此时，模型在内存中。接下来，您需要将模型工件保存到云存储位置。

In [ ]:
DEPLOY_COMPUTE = "n1-standard-4"
print("Deploy machine type", DEPLOY_COMPUTE)

## 从TensorFlow Hub获取预训练模型

为了演示目的，本教程使用了从TensorFlow Hub（TFHub）获取的预训练模型，然后将其上传到“Vertex AI模型”资源。一旦您拥有了“Vertex AI模型”资源，该模型就可以部署到“Vertex AI端点”资源上。

### 下载预训练模型

首先，您需要从TensorFlow Hub下载预训练模型。该模型将作为TF.Keras层下载。为了完成模型，您将在本例中创建一个`Sequential()`模型，其中包含下载的TFHub模型作为一个层，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 32, 32, 3])

tfhub_model.summary()

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

### 将TensorFlow Hub模型上传到`Vertex AI Model`资源

最后，您可以使用`upload()`方法将来自TFHub模型的模型工件上传到`Vertex AI Model`资源，参数如下：

- `display_name`：`Model`资源的易读名称。
- `artifact_uri`：模型包的Cloud Storage位置。
- `serving_container_image_uri`：提供容器映像。

将模型上传到Vertex AI Model资源会返回一个长时间运行的操作，因为这可能需要一些时间。

*注：*当您将模型工件上传到`Vertex AI Model`资源时，必须指定相应的部署容器映像。

In [ ]:
blessed_model = aiplatform.Model.upload(
    display_name="resnet",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    is_default_version=True,
    version_aliases=["v1"],
)

print(blessed_model)

### 创建模型评估

首先，您需要按照预定义的模型评估模式之一创建一个模型评估。在这个例子中，您将使用分类度量模式，并指定以下评估指标的子集作为字典：

- `logLoss`：对数损失。
- `auPrc`：准确率。

然后，您要使用以下参数构建`ModelEvaluation`对象：

- `display_name`：评估指标的可读名称。
- `metrics_schema_uri`：特定类型评估指标的模式。
- `metrics`：包含评估指标的字典。

了解更多关于[评估指标模式](https://cloud.google.com/vertex-ai/docs/evaluation/introduction#features)。

In [ ]:
metrics = {"logLoss": 1.4, "auPrc": 0.85}
print(metrics)

blessed_eval = gapic.ModelEvaluation(
    display_name="eval",
    metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    metrics=metrics,
)

### 上传评估指标到模型注册表

接下来，将从自定义训练作业中的模型评估上传到 Vertex AI 模型注册表中相应的条目。

目前，在 SDK 中还没有对此方法的支持。相反，您需要使用较低级别的 GAPIC API 接口。

In [ ]:
API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"
client = gapic.ModelServiceClient(client_options={"api_endpoint": API_ENDPOINT})

client.import_model_evaluation(
    parent=blessed_model.resource_name, model_evaluation=blessed_eval
)

创建`Endpoint`资源

您可以使用`Endpoint.create()`方法创建`Endpoint`资源。至少，您需要指定终端的显示名称。可选地，您可以指定项目和位置（区域）；否则，设置将继承您使用`init()`方法初始化Vertex AI SDK时设置的值。

在此示例中，指定了以下参数：

- `display_name`：`Endpoint`资源的可读名称。
- `project`：您的项目ID。
- `location`：您的区域。

此方法将返回一个`Endpoint`对象。

了解有关[Vertex AI终端](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)的更多信息。

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="resnet", project=PROJECT_ID, location=REGION
)

print(endpoint)

### 将`Model`资源部署到`Endpoint`资源

接下来，将经过批准的`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源。`Vertex AI Model`资源已经为其定义了部署容器镜像。要部署，您需要指定以下额外的配置设置：

- 机器类型。
- （如果有的话）GPU的类型和数量。
- VM实例的静态、手动或自动缩放。

在这个例子中，您将使用指定参数的最小数量部署模型，如下：

- `model`：`Model`资源。
- `deployed_model_displayed_name`：部署模型实例的人类可读名称。
- `machine_type`：每个VM实例的机器类型。

由于需要准备资源，这可能需要几分钟时间。

In [ ]:
response = endpoint.deploy(
    model=blessed_model,
    deployed_model_display_name="resnet",
    machine_type=DEPLOY_COMPUTE,
)

print(response)

## 为管道创建自定义组件

接下来，您将创建几个自定义组件，用于在管道中使用。

### 创建组件以上传模型的下一个版本

首先，您定义一个组件，将训练好的挑战者模型作为版本上传到`Vertex AI模型注册表`中的受认可模型。此组件接受以下参数：

- `parent_model`：受认可模型的完整资源名称。
- `artifact_uri`：挑战者模型的模型 artifacts 的 Cloud Storage 位置。
- `serving_container`：挑战者模型的服务容器。
- `project`：您的项目ID。
- `region`：您的区域。

In [ ]:
@component(packages_to_install=["google-cloud-aiplatform"])
def create_next_model_version(
    parent_model: str,
    artifact_uri: str,
    serving_container: str,
    project: str,
    region: str,
) -> str:
    from google.cloud import aiplatform

    aiplatform.init(project=project, location=region)

    model = aiplatform.Model.upload(
        display_name="resnet",
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container,
        parent_model=parent_model,
        is_default_version=True,
        version_aliases=["v2"],
        version_description="This is the second version of the model",
    )

    return model.resource_name

### 创建组件来导入分类指标

接下来，您定义一个组件，将评估挑战者模型的指标导入到模型注册表中。该组件接受以下参数：

- `display_name`：评估指标的人类可读名称
- `metrics`：格式化为分类的评估指标
- `parent_model_resource`：挑战者模型版本的完整资源名称
- `region`：区域。

In [ ]:
@component(packages_to_install=["google-cloud-aiplatform"])
def import_classification_metrics(
    display_name: str, metrics: dict, parent_model_resource: str, region: str
):
    from google.cloud.aiplatform import gapic

    evaluation = gapic.ModelEvaluation(
        display_name=display_name,
        metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
        metrics=metrics,
    )

    API_ENDPOINT = f"{region}-aiplatform.googleapis.com"
    client = gapic.ModelServiceClient(client_options={"api_endpoint": API_ENDPOINT})
    client.import_model_evaluation(
        parent=parent_model_resource, model_evaluation=evaluation
    )

创建组件以比较指标

接下来，您可以定义一个组件来比较模型的两个版本之间的 `auPrc` 指标。在这种情况下，当前的核准版本和新的挑战者版本。其中`auPrc` 值最高的模型被设置为默认模型。在随后部署时，默认模型将被部署。该组件接收以下参数：

- `blessed_model_resource_name`：核准模型的完整资源名称。
- `challenger_model_resource_name`：挑战者模型的完整资源名称。

In [ ]:
@component(packages_to_install=["google-cloud-aiplatform"])
def compare_metrics(
    blessed_model_resource_name: str, challenger_model_resource_name: str
):
    from google.cloud import aiplatform

    # Get the metrics for the blessed model
    blessed_model = aiplatform.Model(blessed_model_resource_name)
    blessed_eval = blessed_model.list_model_evaluations()[0]
    blessed_auPrc = blessed_eval.metrics["auPrc"]

    # Get the metrics for the challenger model
    challenger_model = aiplatform.Model(challenger_model_resource_name)
    challenger_eval = challenger_model.list_model_evaluations()[0]
    challenger_auPrc = challenger_eval.metrics["auPrc"]

    # Which model has the best accuracy becomes the default model
    if challenger_auPrc > blessed_auPrc:
        challenger_model.versioning_registry.add_version_aliases(
            new_aliases=["default"], version=challenger_model.version_id
        )
    else:
        blessed_model.versioning_registry.add_version_aliases(
            new_aliases=["default"], version=blessed_model.version_id
        )

## 构建祝福版与挑战者管道

接下来，您构建以下任务的管道：

- 获取模型的祝福版本。
- 获取部署的祝福模型的端点。
- 训练（伪造的）挑战者模型。
- 将挑战者模型上传为模型的下一个版本。
- 导入挑战者模型的评估指标。
- 比较祝福版和挑战者的评估指标，并相应设置默认值。
- 导入现有的生产端点。
- 将新的默认模型部署到生产端点。

In [ ]:
@kfp.dsl.pipeline(name="blessed-vs-challenger")
def pipeline(
    blessed_model_resource: str,
    serving_container: str,
    machine_type: str,
    endpoint_resource_name: str,
    endpoint_resource_uri: str,
    project: str = PROJECT_ID,
    region: str = REGION,
):
    from google_cloud_pipeline_components.experimental.evaluation import \
        GetVertexModelOp
    from google_cloud_pipeline_components.types import artifact_types
    from google_cloud_pipeline_components.v1.endpoint import ModelDeployOp
    from kfp.v2.components import importer_node

    # Get the Vertex AI model resource of the blessed model
    model = GetVertexModelOp(model_resource_name=blessed_model_resource)

    # pretend that you trained a new version of the model (artifacts at MODEL_DIR)

    # create the next version in the Model Registry as the challenger model
    next_version = create_next_model_version(
        parent_model=blessed_model_resource,
        artifact_uri=MODEL_DIR,
        serving_container=serving_container,
        project=project,
        region=region,
    ).after(model)

    # pretend to evaluate the challenger
    challenger_metrics = {"logLoss": 1.3, "auPrc": 0.88}

    # upload the metrics for the challenger version
    import_metrics = import_classification_metrics(
        display_name="challenger",
        metrics=challenger_metrics,
        parent_model_resource=next_version.output,
        region=region,
    ).after(next_version)

    # test metrics
    compare = compare_metrics(blessed_model_resource, next_version.output).after(
        import_metrics
    )

    # import the production Endpoint
    endpoint = importer_node.importer(
        artifact_uri=endpoint_resource_uri,
        artifact_class=artifact_types.VertexEndpoint,
        metadata={"resourceName": endpoint_resource_name},
    )

    # deploy model to endpoint
    _ = ModelDeployOp(
        model=model.outputs["model"],
        endpoint=endpoint.output,  # .outputs["endpoint"],
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
        dedicated_resources_machine_type=machine_type,
        traffic_split={"0": 100},
    ).after(compare)

编译流水线

接下来，你需要编译流水线。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="challenger_vs_blessed.json"
)

### 执行管道

最后，您执行管道，传递以下管道参数值：

- `blessed_model_resource`：模型当前版本的完整资源名称。
- `serving_container`：用于部署模型的serving容器。
- `machine_type`：用于部署模型的机器类型。
- `endpoint_resource_name`：生产端点的完整资源名称。
- `endpoint_resource_uri`：生产端点的完整URI。
- `project`：项目ID。
- `region`：区域。

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/control".format(BUCKET_URI)

job = aiplatform.PipelineJob(
    display_name="challenger_vs_blessed",
    template_path="challenger_vs_blessed.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "blessed_model_resource": blessed_model.resource_name,
        "serving_container": DEPLOY_IMAGE,
        "machine_type": DEPLOY_COMPUTE,
        "endpoint_resource_name": endpoint.resource_name,
        "endpoint_resource_uri": "https://us-central1-aiplatform.googleapis.com/v1/"
        + endpoint.resource_name,
        "project": PROJECT_ID,
        "region": REGION,
    },
    enable_caching=False,
)

job.run()

! rm challenger_vs_blessed.json

获取生产端点的最新状态

现在管道已经完成，挑战者（版本2）模型已经取代了之前在生产端点上的认可模型。

接下来，您显示部署模型的生产端点的最新信息，然后显示流量分割。100%输入的资源ID是挑战者（v2）模型的资源ID。

In [ ]:
gca_resource = endpoint.list(filter="display_name=resnet")[0].gca_resource
print("Deployed Models", gca_resource.deployed_models)
print("\n")
print("Traffic Split", gca_resource.traffic_split)

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除此教程中创建的个别资源。

In [ ]:
import os

# Delete endpoint resource
try:
    endpoint.undeploy_all()
    endpoint.delete()
except Exception as e:
    print(e)

# Delete model resource
try:
    blessed_model.delete()
except Exception as e:
    print(e)

# Delete the pipeline resource
try:
    job.delete()
except Exception as e:
    print(e)

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI